In [24]:
#边界框
import imgaug as ia
import imgaug.augmenters as iaa
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage
import time
import cv2

from PIL import Image
import os
import sys
import random
import imageio

#图像归一化
ia.seed(1)
# image = ia.quokka(size=(256, 256))
# bbs = BoundingBoxesOnImage([
#     BoundingBox(x1=65, y1=100, x2=200, y2=150),
#     BoundingBox(x1=150, y1=80, x2=200, y2=130)
# ], shape=image.shape)

#aug1=iaa.Affine(rotate=45)#右旋转45度
#seq = iaa.Sequential([
aug1=iaa.AdditiveGaussianNoise(scale=0.2*255)#加噪声
aug2=iaa.GaussianBlur(sigma=(0.0, 3.0)) #变模糊
aug3=iaa.AllChannelsCLAHE(clip_limit=(1,10))#图像增强
aug_ro1=iaa.Affine(rotate=-10)#左旋转45度 旋转会有问题 标记框会比原来的大很多
aug_ro2=iaa.Affine(rotate=10)#右旋转45度
aug_ro3=iaa.Affine(rotate=5)#右旋转45度
aug_ro4=iaa.Affine(rotate=-5)
aug_ros = [aug_ro1,aug_ro2,aug_ro3,aug_ro4]

aug6=iaa.Affine(scale=(0.4, 0.7))#Scale images to a value of 50 to 150% of their original size变大
aug7=iaa.Affine(scale=(1.3, 1.6))#Scale images to a value of 50 to 150% of their original size#缩小
aug8=iaa.GammaContrast((2.5, 2.5), per_channel=True)#色相变暗
aug9=iaa.Grayscale(alpha=(0.9))#色相变灰
aug10=iaa.PiecewiseAffine(scale=(0.08))#扭曲图像

# image1,bbs1=aug1(image=image, bounding_boxes=bbs)
# image2,bbs2=aug2(image=image, bounding_boxes=bbs)

# image = Image.open("G:/test-data/1.jpg").convert("RGB")

def boxFromLine(line):
    line = line.strip()
    listFromLine = line.split(' ')        
    cx = float(listFromLine[1]) * ww
    cy = float(listFromLine[2]) * hh
    w = float(listFromLine[3]) * ww
    h = float(listFromLine[4]) * hh
    x1 = cx - w / 2
    y1 = cy - h / 2
    x2 = cx + w / 2
    y2 = cy + h / 2
    return BoundingBox(x1, y1, x2, y2)

def linesFromBoxs(boxs, ww, hh):
    lines = []
    for i in range(boxs.shape[0]):
        x1 = boxs[i][0]
        y1 = boxs[i][1]
        x2 = boxs[i][2]
        y2 = boxs[i][3]
        cx = (x1 + x2) / 2
        cy = (y1 + y2) / 2
        w = x2- x1
        h = y2 - y1
        line = "0 " + '%.6f'%(cx / ww) + " " + '%.6f'%(cy / hh) + " " + '%.6f'%(w / ww) + " " + '%.6f'%(h / hh) + "\n"
        lines.append(line)
    return lines

def clipOutPart(bbs, image_shape):
    bboxs = []
    for box in bbs.bounding_boxes:        
        bboxs.append(box.clip_out_of_image(image_shape));
    bbs2 = BoundingBoxesOnImage(bboxs, image_shape)
    return bbs2
    

# imgFolder = "G:/detect-train/Train-WhiteCell/data/1/"
# imgFolderDst = "G:/detect-train/Train-WhiteCell/data/2/"
imgFolder = "G:/morning-checking/custom-train-yolov3-tiny-hands-open/data/img0/"
imgFolderDst = "G:/morning-checking/custom-train-yolov3-tiny-hands-open/data/img/"

imgsT = os.listdir(imgFolder)
imgs = []
for file in imgsT:
    if file.endswith(".jpg") or file.endswith(".bmp"):
        file2 = file.replace(".jpg", ".txt")
        file2 = file2.replace(".bmp", ".txt")
        if not os.path.exists(os.path.join(imgFolder, file2)):
            continue
        imgInfo = (os.path.join(imgFolder, file), os.path.join(imgFolder, file2))                
        imgs.append(imgInfo)

countImg = 0
for i in range(len(imgs)):
    if i % 100 == 0:
        print("{} / {}".format(i, len(imgs)))
    imgfile = imgs[i][0]
    txtfile = imgs[i][1]
    image = imageio.imread(imgfile)
    hh = image.shape[0]
    ww = image.shape[1]
#     print(image.shape)
#     print(imgfile)
#     print(txtfile)
    fh = open(txtfile)
    lines = fh.readlines()
    fh.close()   
    bboxs = []
    for line in lines:        
        bboxs.append(boxFromLine(line));
    bbs = BoundingBoxesOnImage(bboxs, image.shape)
    for j in range(4):
        countImg+=1
        imgsave = os.path.join(imgFolderDst, str(countImg) + ".jpg")
        txtsave = os.path.join(imgFolderDst, str(countImg) + ".txt")
#         print(txtsave)
        ro_index = j % 4
        aug_ro = aug_ros[ro_index]
        image2, bbs2 = aug1(image=image, bounding_boxes=bbs)
        image2, bbs2 = aug2(image=image2, bounding_boxes=bbs)
        image2, bbs2 = aug_ro(image=image2, bounding_boxes=bbs)
        bbs2 = clipOutPart(bbs2, image.shape)
#         ia.imshow(bbs2.draw_on_image(image2, size=2))
        boxs = bbs2.to_xyxy_array()   
#         print(boxs)
        lines = linesFromBoxs(boxs, ww, hh)
        fh = open(txtsave, 'w')   
#         print(lines)
        fh.writelines(lines)
        fh.close()         
        imageio.imwrite(imgsave, image2)
    




0 / 2072
100 / 2072
200 / 2072
300 / 2072
400 / 2072
500 / 2072
600 / 2072
700 / 2072
800 / 2072
900 / 2072
1000 / 2072
1100 / 2072
1200 / 2072
1300 / 2072
1400 / 2072
1500 / 2072
1600 / 2072
1700 / 2072
1800 / 2072
1900 / 2072
2000 / 2072


In [6]:
file = "fdd.jpg"
file2 = file.replace(".bmp", ".txt")
print(file2)
fff = [1,2]
len(fff)

fdd.jpg


2

In [2]:
import random
import os
import sys

txt_all = "G:/morning-checking/custom-train-yolov3-tiny-hands-open/data/all.txt"
txt_train = "G:/morning-checking/custom-train-yolov3-tiny-hands-open/data/train.txt"
txt_test = "G:/morning-checking/custom-train-yolov3-tiny-hands-open/data/test.txt"

#生成所有图片列表
img_folder = "G:/morning-checking/custom-train-yolov3-tiny-hands-open/data/img"
imgs = os.listdir(img_folder)
fh = open(txt_all, 'w')
for file in imgs:
    if file.endswith(".jpg") or file.endswith(".bmp"):
        fh.write("data/img/" + file + "\n")
fh.close()

#图片拆分成train和test
fh = open(txt_all)
lines = fh.readlines()
random.shuffle(lines)
fh_train = open(txt_train, 'w')
fh_test = open(txt_test, 'w')
for i, line in enumerate(lines):
    if i < len(lines) * 0.2:
        fh_test.write(line)
    else:
        fh_train.write(line)
        
fh_train.close()
fh_test.close()
        
    

